# Imports + Git

In [ ]:
# prompt: ERROR: file:///content/TTS/TTS/tts/layers/xtts does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found

!git clone https://github.com/coqui-ai/TTS
%cd TTS
!pip install -e .[all,dev,notebooks]  # Select the relevant extras

Cloning into 'TTS'...
remote: Enumerating objects: 32844, done.
remote: Counting objects: 100% (2774/2774), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 32844 (delta 2725), reused 2701 (delta 2701), pack-reused 30070 (from 1)
Receiving objects: 100% (32844/32844), 166.14 MiB | 14.02 MiB/s, done.
Resolving deltas: 100% (23844/23844), done.
/content/TTS
Obtaining file:///content/TTS
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 51.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.4 MB/s eta 0:00:00
  Preparing metadat

In [ ]:
!pip install numpy==1.23

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 100.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.0
    Uninstalling numpy-1.22.0:
      Successfully uninstalled numpy-1.22.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.22.0 requires numpy==1.22.0; python_version <= "3.10", but you have numpy 1.23.0 which is incompatible.
albucore 0.0.19 requires numpy>=1.24.4, but you have numpy 1.23.0 which is incompatible.
albumentations 1.4.20 requires numpy>=1.24.4, but you have numpy 1.23.0 which is incompatible.
bigframes 1.29.0 requires numpy>=1.24.0, but you have numpy 1.23.0 which is incompatible.
chex 0.1.88 requires numpy>=1.24.1, but you have numpy 1.23.0 which is incompatible.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
holoviews 1.20.0 requires

Testing tokonizer

In [ ]:
from TTS.TTS.tts.layers.xtts.tokenizer import VoiceBpeTokenizer

# Path to the vocab.json file
vocab_path = "/content/vocab.json"

# Initialize the tokenizer
tokenizer = VoiceBpeTokenizer(vocab_path)


In [ ]:
# Text to tokenize
text = "صباح الخير"

# Tokenize the text (encode it)
tokens = tokenizer.encode(text, lang="ar")

# Print tokenized output
print("Tokenized Output:", tokens)


Tokenized Output: [5022, 3961, 4066, 3953, 2, 4268, 4105]


# Importing Dataset from Google drive

downloading dataset from google drive

In [ ]:
%cd /content

/content


In [ ]:
!gdown --id 1-EB55BAs92xLQa_sM3PptWZ6xlJ6yZhY

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1-EB55BAs92xLQa_sM3PptWZ6xlJ6yZhY
From (redirected): https://drive.google.com/uc?id=1-EB55BAs92xLQa_sM3PptWZ6xlJ6yZhY&confirm=t&uuid=6525b4d3-7f5a-40b6-9c62-adb9002f0923
To: /content/txt_to_speech_dataset.zip
100% 15.6G/15.6G [01:58<00:00, 132MB/s]


In [ ]:
import zipfile
import os

# Create the target directory
target_dir = '/content/lJspeech'
os.makedirs(target_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile("/content/txt_to_speech_dataset.zip", 'r') as zip_ref:
    zip_ref.extractall(target_dir)

print(f'File successfully unzipped to {target_dir}')


File successfully unzipped to /content/lJspeech


# Model Config + Training

In [ ]:
import os

from trainer import Trainer, TrainerArgs

from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig, XttsAudioConfig
from TTS.utils.manage import ModelManager

# Logging parameters
RUN_NAME = "GPT_XTTS_v2.0_LJSpeech_FT"
PROJECT_NAME = "XTTS_trainer"
DASHBOARD_LOGGER = "tensorboard"
LOGGER_URI = None

# Set here the path that the checkpoints will be saved. Default: ./run/training/
OUT_PATH = "/content/run/training/"

# Training Parameters
OPTIMIZER_WD_ONLY_ON_WEIGHTS = True  # for multi-gpu training please make it False
START_WITH_EVAL = True  # if True it will star with evaluation
BATCH_SIZE = 3  # set here the batch size
GRAD_ACUMM_STEPS = 84  # set here the grad accumulation steps
# Note: we recommend that BATCH_SIZE * GRAD_ACUMM_STEPS need to be at least 252 for more efficient training. You can increase/decrease BATCH_SIZE but then set GRAD_ACUMM_STEPS accordingly.

# Define here the dataset that you want to use for the fine-tuning on.
config_dataset = BaseDatasetConfig(
    formatter="ljspeech",
    dataset_name="ljspeech",
    path="/content/lJspeech/",
    meta_file_train="/content/lJspeech/metadata.csv",
    language="ar",
)

# Add here the configs of the datasets
DATASETS_CONFIG_LIST = [config_dataset]


In [ ]:

# Define the path where XTTS v2.0.1 files will be downloaded
CHECKPOINTS_OUT_PATH = os.path.join(OUT_PATH, "XTTS_v2.0_original_model_files/")
os.makedirs(CHECKPOINTS_OUT_PATH, exist_ok=True)


# DVAE files
DVAE_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth"
MEL_NORM_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/mel_stats.pth"

# Set the path to the downloaded files
DVAE_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(DVAE_CHECKPOINT_LINK))
MEL_NORM_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(MEL_NORM_LINK))

# download DVAE files if needed
if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > Downloading DVAE files!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=False)


# Download XTTS v2.0 checkpoint if needed
TOKENIZER_FILE_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/vocab.json"
XTTS_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth"

# XTTS transfer learning parameters: You we need to provide the paths of XTTS model checkpoint that you want to do the fine tuning.
TOKENIZER_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(TOKENIZER_FILE_LINK))  # vocab.json file
XTTS_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(XTTS_CHECKPOINT_LINK))  # model.pth file

# download XTTS v2.0 files if needed
if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
    print(" > Downloading XTTS v2.0 files!")
    ModelManager._download_model_files(
        [TOKENIZER_FILE_LINK, XTTS_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=False
    )


# Training sentences generations
SPEAKER_REFERENCE = [
    "/content/lJspeech/wavs/audio_10302.wav"  # speaker reference to be used in training test sentences
]
LANGUAGE = config_dataset.language




In [ ]:
def main():
    # init args and config
    model_args = GPTArgs(
        max_conditioning_length=132300,  # 6 secs
        min_conditioning_length=66150,  # 3 secs
        debug_loading_failures=False,
        max_wav_length=255995,  # ~11.6 seconds
        max_text_length=200,
        mel_norm_file=MEL_NORM_FILE,
        dvae_checkpoint=DVAE_CHECKPOINT,
        xtts_checkpoint=XTTS_CHECKPOINT,  # checkpoint path of the model that you want to fine-tune
        tokenizer_file=TOKENIZER_FILE,
        gpt_num_audio_tokens=1026,
        gpt_start_audio_token=1024,
        gpt_stop_audio_token=1025,
        gpt_use_masking_gt_prompt_approach=True,
        gpt_use_perceiver_resampler=True,
    )
    # define audio config
    audio_config = XttsAudioConfig(sample_rate=44100, dvae_sample_rate=22050, output_sample_rate=24000)
    # training parameters config
    config = GPTTrainerConfig(
        output_path=OUT_PATH,
        model_args=model_args,
        run_name=RUN_NAME,
        project_name=PROJECT_NAME,
        run_description="""
            GPT XTTS training
            """,
        dashboard_logger=DASHBOARD_LOGGER,
        logger_uri=LOGGER_URI,
        audio=audio_config,
        batch_size=BATCH_SIZE,
        batch_group_size=48,
        eval_batch_size=BATCH_SIZE,
        num_loader_workers=8,
        eval_split_max_size=256,
        print_step=50,
        plot_step=100,
        log_model_step=1000,
        save_step=10000,
        save_n_checkpoints=1,
        save_checkpoints=True,
        # target_loss="loss",
        print_eval=False,
        # Optimizer values like tortoise, pytorch implementation with modifications to not apply WD to non-weight parameters.
        optimizer="AdamW",
        optimizer_wd_only_on_weights=OPTIMIZER_WD_ONLY_ON_WEIGHTS,
        optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
        lr=5e-06,  # learning rate
        lr_scheduler="MultiStepLR",
        # it was adjusted accordly for the new step scheme
        lr_scheduler_params={"milestones": [50000 * 18, 150000 * 18, 300000 * 18], "gamma": 0.5, "last_epoch": -1},
        test_sentences=[
            {
                "text": "صباح الخير انا بتكلم مصرى عادى",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "اهلا انا محمود و دا المشروع",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
        ],
    )

    # init the model from config
    model = GPTTrainer.init_from_config(config)

    # load training samples
    train_samples, eval_samples = load_tts_samples(
        DATASETS_CONFIG_LIST,
        eval_split=True,
        eval_split_max_size=config.eval_split_max_size,
        eval_split_size=config.eval_split_size,
    )

    # init the trainer and 🚀
    trainer = Trainer(
        TrainerArgs(
            restore_path=None,  # xtts checkpoint is restored via xtts_checkpoint key so no need of restore it using Trainer restore_path parameter
            skip_train_epoch=False,
            start_with_eval=START_WITH_EVAL,
            grad_accum_steps=GRAD_ACUMM_STEPS,
        ),
        config,
        output_path=OUT_PATH,
        model=model,
        train_samples=train_samples,
        eval_samples=eval_samples,
    )
    trainer.fit()


if __name__ == "__main__":
    main()

>> DVAE weights restored from: /content/run/training/XTTS_v2.0_original_model_files/dvae.pth


 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: False
 | > Precision: float32
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 1
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False


 | > Found 38158 files in /content/lJspeech


 > Start Tensorboard: tensorboard --logdir=/content/run/training/GPT_XTTS_v2.0_LJSpeech_FT-January-05-2025_06+50AM-0000000

 > Model has 518442047 parameters

 > EPOCH: 0/1000
 --> /content/run/training/GPT_XTTS_v2.0_LJSpeech_FT-January-05-2025_06+50AM-0000000


 > Filtering invalid eval samples!!


...

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.

  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.09169619784635656 (+0)
     | > avg_loss_text_ce: 0.04040187088882221 (+0)
     | > avg_loss_mel_ce: 3.3352582314435173 (+0)
     | > avg_loss: 3.375660099702723 (+0)


 > EPOCH: 1/1000
 --> /content/run/training/GPT_XTTS_v2.0_LJSpeech_FT-January-05-2025_06+50AM-0000000
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings

 > Sampling by language: dict_keys(['ar'])



   --> TIME: 2025-01-05 06:50:48 -- STEP: 0/12634 -- GLOBAL_STEP: 0
     | > loss_text_ce: 0.042523689568042755  (0.042523689568042755)
     | > loss_mel_ce: 3.792584180831909  (3.792584180831909)
     | > loss: 0.04565604776144028  (0.04565604776144028)
     | > current_lr: 5e-06 
     | > step_time: 0.8857  (0.8856673240661621)
     | > loader_time: 5.0869  (5.086939096450806)


   --> TIME: 2025-01-05 06:51:16 -- STEP: 50/12634 -- GLOBAL_STEP: 50
     | > loss_text_ce: 0.03893004730343819  (0.040669028237462025)
     | > loss_mel_ce: 3.3096044063568115  (3.5323894834518432)
     | > loss: 0.039863504469394684  (0.04253641158342361)
     | > current_lr: 5e-06 
     | > step_time: 0.3447  (0.33841997623443604)
     | > loader_time: 0.0117  (0.02016297340393066)


   --> TIME: 2025-01-05 06:51:45 -- STEP: 100/12634 -- GLOBAL_STEP: 100
     | > loss_text_ce: 0.04068758711218834  (0.04070717386901377)
     | > loss_mel_ce: 3.6755995750427246  (3.5176886749267577)
     | > loss: 0.044241

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.10481043422923368 (+0.01311423638287712)
     | > avg_loss_text_ce: 0.030653012193301146 (-0.009748858695521063)
     | > avg_loss_mel_ce: 1.9824791108860689 (-1.3527791205574484)
     | > avg_loss: 2.0131321318009334 (-1.3625279679017894)

 > BEST MODEL : /content/run/training/GPT_XTTS_v2.0_LJSpeech_FT-January-05-2025_06+50AM-0000000/best_model_12634.pth

 > EPOCH: 2/1000
 --> /content/run/training/GPT_XTTS_v2.0_LJSpeech_FT-January-05-2025_06+50AM-0000000

 > TRAINING (2025-01-05 09:05:32) 

   --> TIME: 2025-01-05 09:05:45 -- STEP: 16/12634 -- GLOBAL_STEP: 12650
     | > loss_text_ce: 0.03241506591439247  (0.030542486463673413)
     | > loss_mel_ce: 1.9350053071975708  (2.2210952788591385)
     | > loss: 0.023421671241521835  (0.02680521155707538)
     | > current_lr: 5e-06 
     | > step_time: 0.3474  (0.35421593487262726)
     | > loader_time: 0.0096  (0.018887430429458618)


   --> TIME: 2025-01-05 09:06:16 -- STEP: 66/12634 -- G

# Uploading Training weights to drive

In [ ]:
# prompt: save this file in google drive
from google.colab import drive
drive.mount('/content/drive')

drive.mount("/content/drive", force_remount=True)
!mkdir -p /content/drive/MyDrive/weights/
#!cp -r /content/txt_to_speech_dataset /content/drive/MyDrive/txt_to_speech_dataset_files
!cp /content/run/training/GPT_XTTS_v2.0_LJSpeech_FT-January-05-2025_06+50AM-0000000/best_model.pth /content/drive/MyDrive/weights/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted at /content/drive


#Model Inference

In [7]:
%cd /content/Finetuned_model
!gdown --id 1-131EwOwmJtgC5tg0B9CiGOpN0f1yOGz #checkpoint.pth
!gdown --id 1I38OqwNPx7ikwi-MoRhPjklUBJS-Q20d #config.json

/content/Finetuned_model
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1-131EwOwmJtgC5tg0B9CiGOpN0f1yOGz
From (redirected): https://drive.google.com/uc?id=1-131EwOwmJtgC5tg0B9CiGOpN0f1yOGz&confirm=t&uuid=54e16da6-fab4-425d-96f2-9fcf1bb4a636
To: /content/Finetuned_model/checkpoint_18347.pth
100% 5.61G/5.61G [01:21<00:00, 69.2MB/s]


Original Model

In [ ]:
import os
import torch
import torchaudio
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts

# Add here the xtts_config path
CONFIG_PATH = "/content/run/training/GPT_XTTS_v2.0_LJSpeech_FT-January-05-2025_06+50AM-0000000/config.json"
# Add here the vocab file that you have used to train the model
TOKENIZER_PATH = "/content/run/training/XTTS_v2.0_original_model_files/vocab.json"
# Add here the checkpoint that you want to do inference with
XTTS_CHECKPOINT = "/content/run/training/XTTS_v2.0_original_model_files/model.pth"
# Add here the speaker reference
SPEAKER_REFERENCE = "/content/lJspeech/wavs/audio_10302.wav"

# output wav path
OUTPUT_WAV_PATH = "Original.wav"

print("Loading model...")
config = XttsConfig()
config.load_json(CONFIG_PATH)
model = Xtts.init_from_config(config)
model.load_checkpoint(config, checkpoint_path=XTTS_CHECKPOINT, vocab_path=TOKENIZER_PATH, use_deepspeed=False)
model.cuda()

print("Computing speaker latents...")
gpt_cond_latent, speaker_embedding = model.get_conditioning_latents(audio_path=[SPEAKER_REFERENCE])



Loading model...


/content/TTS/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)


Computing speaker latents...
Inference...


Finetuned Model

In [ ]:
import os
import torch
import torchaudio
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts

# Add here the xtts_config path
CONFIG_PATH = "/content/run/training/GPT_XTTS_v2.0_LJSpeech_FT-January-05-2025_06+50AM-0000000/config.json"
# Add here the vocab file that you have used to train the model
TOKENIZER_PATH = "/content/run/training/XTTS_v2.0_original_model_files/vocab.json"
# Add here the checkpoint that you want to do inference with
XTTS_CHECKPOINT = "/content/run/training/GPT_XTTS_v2.0_LJSpeech_FT-January-05-2025_06+50AM-0000000/checkpoint_18347.pth"
# Add here the speaker reference
SPEAKER_REFERENCE = "/content/lJspeech/wavs/audio_10302.wav"

# output wav path
OUTPUT_WAV_PATH1 = "Finetuned.wav"

print("Loading model...")
config = XttsConfig()
config.load_json(CONFIG_PATH)
model1 = Xtts.init_from_config(config)
model1.load_checkpoint(config, checkpoint_path=XTTS_CHECKPOINT, vocab_path=TOKENIZER_PATH, use_deepspeed=False)
model1.cuda()

print("Computing speaker latents...")
gpt_cond_latent, speaker_embedding = model1.get_conditioning_latents(audio_path=[SPEAKER_REFERENCE])


Loading model...


/content/TTS/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)


Computing speaker latents...
Inference...


In [ ]:
OUTPUT_WAV_PATH1 = "Finetuned.wav"


In [ ]:
OUTPUT_WAV_PATH = "Original.wav"

Testing FineTuned VS Original Model

In [ ]:
sentence =  "الصبح وأنا نازل من البيت لقيت الجو برد جدًا، فاضطريت أرجع أخد جاكيت، ولما وصلت للمحطة اكتشفت إن الميكروباصات كلها زحمة والناس واقفة مستنية"
#Test
gpt_cond_latent, speaker_embedding = model.get_conditioning_latents(audio_path=[SPEAKER_REFERENCE])
out = model.inference(
   sentence,
    "ar",
    gpt_cond_latent,
    speaker_embedding,
    temperature=0.7, # Add custom parameters here
)
torchaudio.save(OUTPUT_WAV_PATH, torch.tensor(out["wav"]).unsqueeze(0), 24000)

out1 = model1.inference(
    sentence,
    "ar",
    gpt_cond_latent,
    speaker_embedding,
    temperature=0.7, # Add custom parameters here
)
torchaudio.save(OUTPUT_WAV_PATH1, torch.tensor(out1["wav"]).unsqueeze(0), 24000)

